In [1]:
import tensorflow as tf
import tensorflow_hub as hub

custom_objects = {'KerasLayer': hub.KerasLayer}

model_path = "galeffnet.h5"  

final_model = tf.keras.models.load_model(model_path,custom_objects)


In [2]:
import pandas as pd
import h5py
import to_rgb
import numpy as np 
file3=h5py.File('test_set.h5','r')
img3=file3['processed_images']

import matplotlib.pyplot as plt
original_length=152
target_length=64
left = (original_length - target_length) // 2
top = (original_length - target_length) // 2
right = left + target_length
bottom = top + target_length

cropped_images = []
for image_array in img3:
    cropped_image = image_array[:, top:bottom, left:right]
    cropped_images.append(cropped_image)


img_rgb3=[to_rgb.dr2_rgb(i,['g','r','z'])  for i in cropped_images]
img_rgb3=np.array(img_rgb3)

In [3]:
ssfr_true=np.array(file3['specsfr_fib_p50']).reshape(-1,1)
lgm_true=np.array(file3['lgm_fib_p50']).reshape(-1,1)

In [4]:
import pickle 
with open('scaler_sfr.pkl', 'rb') as file:
    scaler_sfr = pickle.load(file)

with open('scaler_lgm.pkl', 'rb') as file:
    scaler_lgm = pickle.load(file)


In [5]:
y=final_model.predict(np.array(img_rgb3))
lgm_pred=np.array(y[0])
lgm_pred=scaler_lgm.inverse_transform(lgm_pred)

sfr_pred=np.array(y[1])
sfr_pred=scaler_sfr.inverse_transform(sfr_pred)

In [6]:
df=pd.DataFrame()
df['ssfr']=pd.DataFrame(sfr_pred)
df['lgm']=pd.DataFrame(lgm_pred)

In [7]:
def count_mae(n1,n2):
    n1=np.array(n1)
    n2=np.array(n2)
    absolute_errors = np.abs(n1 - n2)

    mae = np.mean(absolute_errors)

    print("MAE:", mae)

In [8]:
print("MAE for lgm:")
count_mae(lgm_true,lgm_pred)

MAE for lgm:
MAE: 0.14751796854267374


In [10]:
sd = np.std(np.array(lgm_true) - np.array(lgm_pred))
mse = np.mean((np.array(lgm_true) - np.array(lgm_pred)) ** 2)
rmse=np.sqrt(mse)
print(f'LGM: MSE：{mse},RMSE：{rmse},SD：{sd}')

LGM: MSE：0.04924270269524266,RMSE：0.22190696855944533,SD：0.2178983054407689


In [12]:
import numpy as np

true_values = lgm_true
predicted_values = df['lgm']
count_condition_met = 0

for real, predicted in zip(true_values, predicted_values):
    #ratio = predicted / real
    
    if abs(real-predicted) > 0.3:
        count_condition_met += 1

percentage_condition_met = (count_condition_met / len(true_values)) 

print(f"LGM fout：{percentage_condition_met}")



LGM fout：0.1293


In [25]:
true_values = np.array(lgm_true)
true_values = true_values.reshape(-1)
predicted_values = np.array(df['lgm'])

m=np.median((predicted_values-true_values))
print(f"LGM median：{m}")

LGM median：0.022099506835937532


In [33]:
print("MAE for ssfr:")
count_mae(ssfr_true,sfr_pred)

MAE for ssfr:
MAE: 0.29499949855242713


In [30]:
sd = np.std(np.array(ssfr_true) - np.array(sfr_pred))
mse = np.mean((np.array(ssfr_true) - np.array(sfr_pred)) ** 2)
rmse=np.sqrt(mse)
print(f'sSFR MSE:{mse},RMSE:{rmse},SD:{sd}')

sSFR MSE:0.17055504599679752,RMSE:0.41298310618813155,SD:0.40985196449311884


In [31]:
import numpy as np

true_values = ssfr_true
predicted_values = df['ssfr']
count_condition_met = 0

for real, predicted in zip(true_values, predicted_values):
    #ratio = predicted / real
    
    if abs(real-predicted) > 0.3:
        count_condition_met += 1

percentage_condition_met = (count_condition_met / len(true_values)) 

print(f"sSFR fout：{percentage_condition_met}")



sSFR fout：0.3595333333333333


In [32]:
true_values = np.array(ssfr_true)
true_values = true_values.reshape(-1)
predicted_values = np.array(df['ssfr'])

m=np.median((predicted_values-true_values))
print(f"sSFR median：{m}")

sSFR median：-0.015964782104492592
